## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [9]:
data = list(range(1, 1001))
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [4]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [12]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [13]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [14]:
# count
rdd.count()

1000

In [15]:
# first
rdd.first()

1

In [16]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [17]:
# top
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [18]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [19]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([6.36937594e-01, 4.22701178e-01, 8.05424452e-01, 2.87261767e+00,
       1.19291872e+00, 2.69997663e+00, 2.86597123e+00, 2.65378764e+00,
       7.06006601e-02, 3.27241405e+00, 5.18500535e-01, 7.60070720e-01,
       5.90178524e+00, 1.31132946e+01, 1.22882918e+01, 4.88849750e+00,
       3.17284545e+00, 7.04980365e+00, 1.80734903e+00, 3.03353478e+00,
       1.95282438e+01, 1.17777868e+01, 2.28953986e+01, 8.12674903e+00,
       1.93520438e+01, 6.30337149e-01, 1.19350141e+01, 1.13039976e+01,
       2.06254454e+01, 5.83407185e+00, 2.73262244e+01, 1.78339891e+01,
       3.23633679e+01, 2.47138076e+01, 7.71798704e+00, 2.81376901e+01,
       2.98926506e+01, 1.64865624e+01, 7.81105125e+00, 3.68285811e+00,
       1.89068182e+01, 1.87542589e-01, 2.23778424e+01, 4.37730789e+01,
       3.25959781e+01, 5.39183440e+00, 3.29397908e+01, 2.96067734e+00,
       2.68637110e+00, 2.05680102e+01, 5.00090103e+01, 4.06197155e+01,
       8.98277810e-01, 1.58156782e+01, 6.58735476e+00, 4.18608054e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [20]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(10)

[0.6369375935365493,
 0.42270117798830764,
 0.805424452442603,
 2.8726176687044944,
 1.1929187162191464,
 2.6999766306713067,
 2.8659712348971658,
 2.653787640203351,
 0.07060066006409926,
 3.2724140493853704]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [21]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [22]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.5859825860536254,
 0.388885083749243,
 0.7409904962471947,
 2.642808255208135,
 1.0974852189216147,
 2.483978500217602,
 2.6366935361053927,
 2.4414846289870833,
 0.06495260725897133,
 3.010620925434541]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [51]:
discounted = revenue_minus_tax.map(lambda x: x * 0.9)

In [52]:
discounted.take(10)

[0.5273843274482629,
 0.3499965753743187,
 0.6668914466224752,
 2.378527429687322,
 0.9877366970294532,
 2.2355806501958417,
 2.3730241824948535,
 2.197336166088375,
 0.058457346533074196,
 2.7095588328910867]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [37]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[792.692964579938,
 772.2243915207783,
 760.0135196466008,
 743.9059968889726,
 735.8515035002195,
 730.1140765095479,
 725.0769977901894,
 722.0645713015081,
 716.4988421961733,
 713.559912105864,
 704.6353534707079,
 697.4451496692818,
 695.6671011888646,
 688.2459861455576,
 681.2463594660219]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [38]:
discounted.toDebugString()

b'(10) PythonRDD[26] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[7] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [43]:
mapped = price_items.map(lambda x: (x, x * 0.9 * 0.92))
print(mapped.count())
print(mapped.take(10))

1000
[(0.6369375935365493, 0.5273843274482628), (0.42270117798830764, 0.3499965753743187), (0.805424452442603, 0.6668914466224753), (2.8726176687044944, 2.378527429687322), (1.1929187162191464, 0.9877366970294532), (2.6999766306713067, 2.235580650195842), (2.8659712348971658, 2.3730241824948535), (2.653787640203351, 2.197336166088375), (0.07060066006409926, 0.0584573465330742), (3.2724140493853704, 2.709558832891087)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [41]:
flat_mapped = price_items.flatMap(lambda x: (x, x * 0.9 * 0.92))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.6369375935365493, 0.5273843274482628, 0.42270117798830764, 0.3499965753743187, 0.805424452442603, 0.6668914466224753, 2.8726176687044944, 2.378527429687322, 1.1929187162191464, 0.9877366970294532]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [53]:
# use the filter function
selected_items = discounted.filter(lambda x: x > 300)

# calculate total remaining in inventory 
selected_items.count()

288

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [54]:
selected_items.reduce(lambda x, y: x + y)

130650.84321060804

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [57]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(29, 306.1445221861773),
 (21, 313.9733622501277),
 (50, 317.1232388234389),
 (19, 303.7428145958361),
 (4, 332.2382649763625),
 (28, 308.82358943926),
 (11, 303.5443187348771)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [60]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x+y)
total_spent.take(10)

[(10, 3531.5650854112123),
 (20, 2054.596410337823),
 (40, 4073.6575595803693),
 (50, 3617.8821068894795),
 (30, 2119.568628435572),
 (1, 3007.6507642203132),
 (31, 4259.495488128333),
 (11, 2081.1917681012023),
 (21, 1409.1311105856455),
 (41, 1669.7086209776649)]

In [67]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(5, 5364.878816020233),
 (6, 4821.557478434619),
 (23, 4613.961468895632),
 (31, 4259.495488128333),
 (40, 4073.6575595803693),
 (24, 4019.376104850323),
 (50, 3617.8821068894795),
 (7, 3572.74554178755),
 (10, 3531.5650854112123),
 (18, 3498.4545096801726),
 (48, 3398.6945875569786),
 (22, 3329.618413982227),
 (35, 3255.5820690003607),
 (38, 3123.1267040135726),
 (42, 3104.297897523975),
 (28, 3061.3442787496597),
 (15, 3020.3772145394755),
 (1, 3007.6507642203132),
 (43, 2999.615648617517),
 (14, 2660.8066270075556),
 (27, 2646.288405185624),
 (9, 2641.098285220316),
 (39, 2587.415149999819),
 (17, 2523.943709376208),
 (46, 2521.94708363676),
 (45, 2512.027755872742),
 (4, 2334.5760785078523),
 (47, 2321.589469702867),
 (32, 2206.3869424699824),
 (26, 2201.0498782466157),
 (29, 2131.4537302015133),
 (30, 2119.568628435572),
 (2, 2097.98072927276),
 (11, 2081.1917681012023),
 (20, 2054.596410337823),
 (12, 2028.1663458180835),
 (8, 1995.068365726249),
 (3, 1840.9455709344884),
 (19, 

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [74]:
total_bought = sales_data.countByKey()
sorted(total_bought.items(), key=lambda x: x[1], reverse=True)

[(14, 12),
 (21, 12),
 (11, 11),
 (18, 10),
 (12, 10),
 (10, 10),
 (29, 9),
 (20, 8),
 (24, 8),
 (48, 8),
 (17, 7),
 (41, 7),
 (28, 7),
 (42, 7),
 (8, 6),
 (33, 6),
 (2, 6),
 (49, 6),
 (40, 6),
 (5, 6),
 (6, 6),
 (46, 6),
 (35, 6),
 (30, 5),
 (25, 5),
 (27, 5),
 (45, 5),
 (4, 5),
 (19, 5),
 (15, 5),
 (39, 5),
 (36, 4),
 (47, 4),
 (1, 4),
 (37, 4),
 (9, 4),
 (26, 4),
 (50, 4),
 (34, 4),
 (3, 4),
 (44, 4),
 (22, 4),
 (31, 4),
 (23, 4),
 (13, 4),
 (16, 4),
 (32, 3),
 (7, 3),
 (43, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
